# **Tutorial 3** - CrewAI Introduction

After getting to know the data, it's time to build our first generative AI solution. In this notebook, we will showcase how we can use the Langchain Bedrock API and build a Gen AI with Claude 3 Haiku. We will gain an understanding of how to use the crewAI framework to build multi-agent systems.

In GDSC, we encourage you to choose CrewAI over LangChain for its simplicity, but you are free to use [LangChain](https://python.langchain.com/v0.2/docs/introduction/) if you prefer. CrewAI is user-friendly and ideal for task management and team collaboration. LangChain, on the other hand, offers more advanced features for developers looking to create complex applications. Ultimately, the choice depends on your specific needs and technical expertise.

In this tutorial, we will focus on CrewAI to help you get started quickly and easily.
We recommend you to familiarize yourself with [CrewAI documentation](https://docs.crewai.com/).

## Agenda
1. [What are AI agents](#what-are-ai-agents) - Gives a broad understanding of ai agents concept in context of crewAI.

2. [Setting up the environment](#setup) - Provides all the essential code required to run this notebook.

3. [How to use Bedrock API](#bedrock-api) - Describes how to use Bedrock API.

4. [CrewAI Example](#simple-crewai-example) - An example of using crewAI for a simple code assistant.
    - [Agents](#agents) - Describes the characteristics and behavior of AI agents in the Crew AI framework.
    - [Tasks](#tasks) - Provides information about how the tasks work in the crewAI framework.
    - [Crew](#crew) - Explains how to assemble an ai crew.
    - [Tools](#tools) - Describes how tools for crewAI and langchain are constructed.
5. [Testing](#testing) - We test our solution here.


## What are AI agents 
AI agents are entities that interact with LLM (Language Model) APIs to generate text based on given prompts. These agents perform specific tasks and have defined roles, expertise, and goals. They can work together in either a sequential or hierarchical process to achieve a common goal, ensuring efficient and structured task completion.

In a sequential process, agents collaborate by passing the output of one agent as the input to the next, creating a linear flow of information and actions. Each agent has a specific role and is assigned tasks related to that role. For example, in the code snippet below, we have two agents: a Python developer and a tester. The Python developer agent is responsible for writing Python code to solve the user's question, while the tester agent ensures the code works correctly. The chaining process starts with the Python developer agent writing the Python code based on the user's prompt. The output of this agent, which is the Python code, is then passed to the tester agent. The tester agent tests the code and provides the test results as the final output.

In a hierarchical process, agents are organized in a tiered structure where higher-level agents oversee and coordinate the actions of lower-level agents. This allows for more complex task management and decision-making. Higher-level agents can delegate subtasks to lower-level agents and integrate their outputs into a cohesive solution. For instance, a project manager agent might oversee both the Python developer and tester agents, ensuring their tasks align with the overall project goals. The project manager agent can evaluate the final output, make necessary adjustments, and provide comprehensive feedback.

## Setup
First, we need to import the required libraries and functions.

In [1]:
!pip install langchain==0.1.20 crewai==0.28.8 langchain-core==0.1.52 langchain-aws==0.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which vers

In [2]:
# Imports
import os
from crewai import Agent, Crew, Process, Task
from crewai.project import agent, crew, task
from langchain_aws import ChatBedrock
from langchain_core.tools import tool

# # Setup environmental variables, for local IDE users only
# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key_id'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_access_key'
# os.environ['AWS_SESSION_TOKEN'] = 'your_session_token'
# os.environ['AWS_REGION'] = 'us-east-1'

## How to use Bedrock API

In GDSC, you can choose from several LLMs that are available on Amazon Bedrock. For the sake of cost reduction and speed, in this tutorial, we will use Claude 3 Haiku.

The Bedrock API provides a powerful platform for building generative AI solutions. It allows us to leverage pre-trained language models like Claude 3 family.

To get started with the Bedrock API, we imported the required libraries and functions. In this tutorial, we will be using the `crewai` library, which provides a high-level interface for working with the Bedrock API. Additionally, we also imported the necessary modules from the `langchain_aws` and `langchain_core.tools` packages.

To use the Bedrock API, you will need to have an API key and a model ID. The model ID represents the specific language model you want to use. In this tutorial, we will be using the Claude 3 Haiku model. You can check model IDs [here](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/models).
![Bedrock Base Models](img/bedrock.png)\
Once you have imported the required libraries and obtained the necessary aws credentials (Tutorial 1), you can create an instance of the `ChatBedrock` class, passing in the model ID and any additional model-specific parameters. This class provides methods for generating text based on given prompts.


In [3]:
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0})

Code below is an example how we can interact with Bedrock LLMs using ChatBedrock. We will not be using it as crewAI provides simple wrapper around this method.

In [5]:
message = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "Hello world!"),
]

test = llm.invoke(message)
print(test)

content='Bonjour le monde !' additional_kwargs={'usage': {'prompt_tokens': 28, 'completion_tokens': 10, 'total_tokens': 38}} response_metadata={'model_id': 'anthropic.claude-3-haiku-20240307-v1:0', 'usage': {'prompt_tokens': 28, 'completion_tokens': 10, 'total_tokens': 38}} id='run-aae8da4e-d62e-4fea-8655-bdc82be2256b-0'


We can analyze this example, especially the first part as it contains a bit of prompt engineering which you will use extensively during GDSC.

In this case this prompt engineering is role assumption, we ask LLM to assume role of translator.

Role assumption is very helpful in stabilization and enhancing of LLMs output.

You can read more on prompt engineering here: [Prompt Engineering guide](https://www.promptingguide.ai/)

# Simple crewAI Example

In the example code provided, we create a `PythonHelpCrew` class that utilizes the Langchain Bedrock API to provide assistance with Python programming. The class defines two agents: a Python developer and a tester. The `pythonDeveloper` agent is responsible for writing Python code to solve the user's question, while the `tester` agent is responsible for testing the code to ensure it works correctly.

The `code_python_task` and `test_code_task` methods define the tasks that the agents will perform. These tasks have descriptions, expected outputs, and references to the corresponding agents.

Finally, the `crew` method creates an instance of the `Crew` class, which represents the group of agents and tasks working together. The `Crew` class allows you to specify the process (sequential or hierarchical), verbosity level, maximum number of iterations, and caching options.

To run the crew and get an answer to the question, you can create an instance of the `PythonHelpCrew` class and call the `run` method, passing in the prompt as a parameter.


Note that showcased code is one of many methods of implementing crewAI. You can check other methods on their [crewAI-examples repository](https://github.com/crewAIInc/crewAI-examples).

In [6]:
class PythonHelpCrew:
    def __init__(self, llm: ChatBedrock) -> None:
        self.llm = llm
    
    def run(self, prompt: str) -> str:
        self.prompt = prompt
        return self.crew().kickoff()

    @agent
    def pythonDeveloper(self) -> Agent:
        return Agent(
            role="Python developer", 
            backstory="Experienced Python developer with deep knowledge in Python programming.", # We do a role assumption technique here
            goal="Write a Python code to solve the user's question.", # The simpler goal the better
            llm=self.llm,
            allow_delegation=False,
            verbose=True)
    
    @agent
    def tester(self) -> Agent:
        return Agent(
            role="Tester",
            backstory="Experienced tester with deep knowledge in testing and using provided tools.", # We do a role assumption technique here and order the agent to provided tool for testing.
            goal="Test the Python code to ensure it works correctly.", # The simpler goal the better
            llm=self.llm,
            allow_delegation=True, # Allow delegation to other agents (python developer), if code fails it will be sent back to the python developer to fix.
            tools = [eval_python_code], # Tools need to be passed in python list format, even if there is only one tool.
            verbose=True)

    
    @task
    def code_python_task(self) -> Task: 
        return Task(
            description=f"Write a python code to solve the user's question: {self.prompt}.", # We format task description with the user prompt passed in the run method.
            expected_output="Python code that solves the user's question. Python code only.", # We specify the expected output of the task. Note that we narrow down response distribution to python code only.
            agent=self.pythonDeveloper()) # Pass appropriate agent to the task.
    @task
    def test_code_task(self) -> Task:
        return Task(
            description="Test the python code to ensure it works correctly.",
            expected_output="Python code only or test results.",
            agent=self.tester())
    
    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=self.agents,  # List of agents participating in the crew. Each agent flagged with @agent decorator will be added here.
            tasks=self.tasks,  # List of tasks to be performed by the agents in the crew. Each task flagged with @task decorator will be added here.
            process=Process.sequential,  # Process type (sequential or hierarchical)
            verbose=2,  # Verbosity level
            max_iter=5,  # Maximum number of repetitions each agent can perform to get the generate the best answer.
            cache=False  # Caching option. Useful when tools produce large output like result of SQL queries.
        )
    

@tool
def eval_python_code(code: str) -> str:
    """
    Evaluate the given Python code and return the result.

    Parameters:
    code (str): The Python code to be executed.

    Returns:
    str: The result of executing the code. If the code executes successfully, it returns "Code executed successfully."
         If an exception occurs during execution, it returns the error message as a string.
    """
    # Remember each tool should have informative docstring. It will be used in the CrewAI platform to provide information about the tool.
    # We recommend generating it with GenAI tools such as Microsoft Copilot.
    try:
        exec(code)
        return "Code executed successfully."
    except Exception as e:
        return str(e)

## Example breakdown

## Agents

Agent arguments are used to define the characteristics and behavior of an agent in the Crew AI framework. 

For the `python_developer` agent, the following arguments are defined:
- `role`: Specifies the role of the agent, which is "Python developer" in this case.
- `backstory`: Provides a description of the agent's background and expertise. It states that the agent is an experienced Python developer with deep knowledge in Python programming.
- `goal`: Defines the agent's goal, which is to write a Python code to solve the user's question.
- `llm`: Specifies the language model to be used by the agent, which is an instance of the `ChatBedrock` class.
- `allow_delegation`: Determines whether the agent can delegate tasks to other agents. In this case, delegation is not allowed.
- `verbose`: Controls the verbosity level of the agent's output. It is set to `True` to enable verbose output.

For the `tester` agent, the following arguments are defined:
- `role`: Specifies the role of the agent, which is "tester".
- `backstory`: In this case the agent is an experienced tester with deep knowledge in testing.
- `goal`: Test the Python code to ensure it works correctly.
- `llm`: `ChatBedrock` class.
- `allow_delegation`: In this case, delegation is allowed. If the code does not pass the test, it will be re-delegated back to the Python developer agent.
- `tools`: Specifies the tools that the agent can use. Agent can use tool if provided but doesn't have to, its up to his decision unless we explicitly order him to do so in `goal` or `backstory` In this case, the `eval_python_code` function is defined as a tool for the tester agent.
- `verbose`: It is set to `True` to enable verbose output.


You can check other agent attributes that may come useful [here](https://docs.crewai.com/core-concepts/Agents/).
## Tasks

Tasks can be defined as the steps an ai crew must take to accomplish a common goal. As LHMs (Large Human Models), we can recognize what steps must be taken in order for the task we define to be accomplished in the best possible way.
Our example ai crew specializes in writing code for python, so what steps must be taken to write code for python?  Write code to python and test it.

For the `code_python_task` task, the following arguments are defined:
- `description` : Short description of the task, for this task it is to write a python code. It also contains original task queried by user.
- `expected_output` : Short description of expected output, here we can stabilize output to our expected form. For this task we want Python code that solves the task, python code only.
- `agent` : Here we forward our previously defined `python_developer` agent.

For the `test_code_task` task, the following arguments are defined:
- `description` : For this task we want our agent to test the code produced by `python_developer` agent.
- `expected_output` : In this case results of testing.
- `agent` : Here we forward our previously defined `tester` agent.

You can check other task attributes that may come useful [here](https://docs.crewai.com/core-concepts/Tasks/).
## Crew

In crew function we put all building blocks together and assemble our crew.
- `agents` : Here we pass our agents, in this implementation framework automatically recognizes agent marked by `@agent` flag, using this flag adds object to global list of agents.
- `tasks` : Similarly to agents, framework recognizes tasks marked by `@task` flag.
- `process` : Here we define inference process. In our example we use `Process.sequential` where crew starts from task specified by us, in our example we defined `code_python_task` first so it will start from it. Another approach implemented by Crew AI framework is `Process.hierarchical`, you can read more about it here [Hierarchical Process](https://docs.crewai.com/how-to/Hierarchical/#implementing-the-hierarchical-process).
- `verbose` : Here we define verbosity for crew output.
- `max_iter` : Here we define number of repetitions each agent can take in solving task. Changing this value has a great impact on balance between thoroughness and efficiency. Once the agent approaches this number, it will try its best to give a good answer.
- `cache` : This argument specifies if crew will use cache to store output from tools. For example if tool produces large output like result of SQL queries, storing it in cache reduces load on external resources and speeds up the execution time.

You can check other crew attributes that may come useful [here](https://docs.crewai.com/core-concepts/Crews/).
## Tools

Tools are python functions that can be used by AI agents. They can perform a whole range of actions such as in this case executing python code, serving as a calculator or connecting to a database. These functions need to have a docstring so that agents can parse what input and output the function takes. These tools can be very general as in this case, or be directed to perform a strongly determined action such as executing a single SQL query. 

In this implementation we mark our `eval_python_code` function with `@tool` flag to add it to tools list.

You can read more on crewAI tools [here](https://docs.crewai.com/core-concepts/Tools/#key-characteristics-of-tools).

## Testing
Now we can test our crew, feel free to play with it. (Just in case try to avoid asking for potentially dangerous code as this crew will execute it during testing)



In [8]:
pythonCrew = PythonHelpCrew(llm=llm)
res = pythonCrew.run("How do I read a file in Python?")
print(res)

2024-08-02 18:45:36,921 - 139924001490752 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Python developer
 [INFO]: == Starting Task: Write a python code to solve the user's question: How do I read a file in Python?.


> Entering new CrewAgentExecutor chain...
Thought: I now have a good understanding of the user's question and the expected criteria for the final answer. I will provide a complete Python code that solves the problem of reading a file in Python.

Final Answer:

```python
# Open a file in read mode
file = open("example.txt", "r")

# Read the entire file
content = file.read()
print(content)

# Read the file line by line
file.seek(0)  # Reset the file pointer to the beginning of the file
for line in file:
    print(line.strip())

# Read a specific number of characters
file.seek(0)  # Reset the file pointer to the beginning of the file
first_10_chars = file.read(10)
print(first_10_chars)

# Close the file
file.close()
```

This Python code demonstrates the following ways to read a file:

1. Reading the entire file content using the `rea

In [9]:

res = pythonCrew.run("How to get n-th fibonacci number?")
print(res)

2024-08-02 18:47:16,396 - 139924001490752 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Python developer
 [INFO]: == Starting Task: Write a python code to solve the user's question: How to get n-th fibonacci number?.


> Entering new CrewAgentExecutor chain...
Thought: I now have a good understanding of the user's question and the expected criteria for the final answer. I will provide a Python code that solves the problem of getting the n-th Fibonacci number.

Final Answer:

```python
def fibonacci(n):
    """
    Calculates the n-th Fibonacci number.
    
    Args:
        n (int): The position of the Fibonacci number to be calculated.
    
    Returns:
        int: The n-th Fibonacci number.
    """
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Example usage
n = 10
print(f"The {n}-th Fibonacci number is: {fibonacci(n)}")
```

This Python code defines a function `fibonacci(n)` that calculates the n-th Fibonacci number. The function uses a recursive approach, where 

In [19]:
print(res)

```python
def fibonacci(n):
    """
    Calculates the n-th Fibonacci number.
    
    Args:
        n (int): The position of the Fibonacci number to be calculated.
    
    Returns:
        int: The n-th Fibonacci number.
    """
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Example usage
n = 10
print(f"The {n}-th Fibonacci number is: {fibonacci(n)}")
```

The provided Python code correctly implements the Fibonacci sequence using a recursive approach. The `fibonacci()` function takes an integer `n` as input and returns the n-th Fibonacci number.

The function works as follows:

1. If `n` is less than or equal to 0, the function returns 0, as the 0th Fibonacci number is defined as 0.
2. If `n` is equal to 1, the function returns 1, as the 1st Fibonacci number is defined as 1.
3. For all other cases, the function recursively calls itself with `n-1` and `n-2` as arguments, and returns the sum of the results.


In [10]:
res = pythonCrew.run("Perceptron class implemented with numpy")

2024-08-02 18:48:48,279 - 139924001490752 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Python developer
 [INFO]: == Starting Task: Write a python code to solve the user's question: Perceptron class implemented with numpy.


> Entering new CrewAgentExecutor chain...
Thought: I can provide a complete Python code implementation of the Perceptron algorithm using NumPy to solve the user's question.

Final Answer:

```python
import numpy as np

class Perceptron:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        """
        Trains the Perceptron model on the input data X and labels y.
        
        Args:
            X (numpy.ndarray): Input data, shape (n_samples, n_features).
            y (numpy.ndarray): Labels, shape (n_samples,).
        """
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        f

In [11]:
print(res)

```python
import numpy as np

class Perceptron:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        """
        Trains the Perceptron model on the input data X and labels y.
        
        Args:
            X (numpy.ndarray): Input data, shape (n_samples, n_features).
            y (numpy.ndarray): Labels, shape (n_samples,).
        """
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.num_iterations):
            for i in range(n_samples):
                prediction = np.dot(X[i], self.weights) + self.bias
                if (y[i] * prediction) <= 0:
                    self.weights += self.learning_rate * y[i] * X[i]
                    self.bias += self.learning_rate * y[i]

    def predict(self, X):
        """

In [25]:
res = pythonCrew.run("Code with syntax error")

2024-08-01 15:26:10,676 - 140633562722304 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Python developer
 [INFO]: == Starting Task: Write a python code to solve the user's question: Code with syntax error.


> Entering new CrewAgentExecutor chain...
Thought: I can provide a Python code that solves the user's question and fixes the syntax error.

Final Answer:

```python
def solve_user_question():
    # Get user input
    user_input = input("Enter a number: ")

    # Convert user input to an integer
    try:
        num = int(user_input)
    except ValueError:
        print("Invalid input. Please enter a number.")
        return

    # Calculate the square of the number
    square = num ** 2

    # Print the result
    print(f"The square of {num} is {square}")

# Call the function to solve the user's question
solve_user_question()
```

This Python code solves the user's question by:

1. Prompting the user to enter a number.
2. Converting the user's input to an integer using the `int()` function, and handling the `ValueError` exception if the inp